In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
df = pd.read_excel('Product list as of 31.12.2021.xlsx')

In [3]:
df.columns=['Product / Underlying','Fee','ISIN','WKN','Currency',
'SIX Swiss Exchange','Deutsche Börse Xetra','Euronext Amsterdam','Euronext Paris','Wiener Börse','Nasdaq Stockholm','BX Swiss','Börse Stuttgart','Börse Düsseldorf']

In [4]:
df = df.iloc[5:]

df.reset_index(drop=True,inplace=True)

df = df[~df['Product / Underlying'].isna()]
df = df[~(df['Product / Underlying']=='\xa0')]

df.reset_index(drop=True,inplace=True)

In [5]:
df

,Product / Underlying,Fee,ISIN,WKN,Currency,SIX Swiss Exchange,Deutsche Börse Xetra,Euronext Amsterdam,Euronext Paris,Wiener Börse,Nasdaq Stockholm,BX Swiss,Börse Stuttgart,Börse Düsseldorf
0,21Shares Algorand ETP,0.025,CH1146882316,A3GVVU,USD,ALGO SE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21Shares Avalanche ETP,0.025,CH1135202088,A3GVVT,CHF,AVAXCHF SE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21Shares Avalanche ETP,0.025,CH1135202088,A3GVVT,EUR,AVAXEUR SE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21Shares Avalanche ETP,0.025,CH1135202088,A3GVVT,USD,AVAX SE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21Shares Binance BNB ETP,0.025,CH0496454155,A22GRU,CHF,,,,,,NaN,ABNB BW,,
5,21Shares Binance BNB ETP,0.025,CH0496454155,A22GRU,EUR,,,,BNBA FP,,NaN,,,21XJ GD
6,21Shares Binance BNB ETP,0.025,CH0496454155,A22GRU,USD,ABNB SE,,BNBA NA,,,NaN,,,
7,21Shares Bitcoin Cash ETP,0.025,CH0475552201,A2126N,CHF,,,,,,NaN,ABCHCHF BW,,
8,21Shares Bitcoin Cash ETP,0.025,CH0475552201,A2126N,EUR,,ABCH GY,,ABCH FP,,NaN,,ABCH GS,ABCH GD
9,21Shares Bitcoin Cash ETP,0.025,CH0475552201,A2126N,USD,ABCH SE,,ABCH NA,,,NaN,,,


In [6]:
df_template = pd.read_excel('Template pre-trade analysis.xlsx', usecols="A:I",
                   sheet_name = "Sheet1",
                   engine='openpyxl')

In [7]:
name = df_template[df_template['Unnamed: 0']=='Name']['Unnamed: 1'].to_list()[0]

isin = df[df['Product / Underlying']== name]['ISIN'].to_list()[0]

In [8]:
# df_template.to_excel('test.xlsx',index=False,header=False)

In [9]:
df_1 = pd.DataFrame({'Name':['Name','ISIN','TER','Type','Underlying'],
             'Info': [name,isin,float('NaN'),'Single Asset','Bitcoin']
             })

In [10]:
df_1

,Name,Info
0,Name,21Shares Bitcoin ETP
1,ISIN,CH0454664001
2,TER,NaN
3,Type,Single Asset
4,Underlying,Bitcoin


In [11]:
ex = df[(df['Product / Underlying']==name) & (df['Currency']=='USD')]['SIX Swiss Exchange'].to_list()[0].split()[0]

In [12]:
df_2 = pd.DataFrame({'Name':['Primary  Exchange','Ticker','Inception date','Last NAV','AUM'],
             'Info': ['SIX',ex,float('NaN'),float('NaN'),float('NaN')]
             })

In [13]:
df_2

,Name,Info
0,Primary Exchange,SIX
1,Ticker,ABTC
2,Inception date,NaN
3,Last NAV,NaN
4,AUM,NaN


In [14]:
d = df[df['Product / Underlying']==name][list(df)[5:]].T

d = d.fillna(' ')

In [15]:
d

,10,11,12,13,14
SIX Swiss Exchange,ABTCCHF SE,ABTCEUR SE,ABTCGBP SE,ABTC SE,
Deutsche Börse Xetra,,2BTC GY,,,
Euronext Amsterdam,,,,ABTC NA,
Euronext Paris,,ABTC FP,,,
Wiener Börse,,ABTC AV,,,
Nasdaq Stockholm,,,,,ABTC SS
BX Swiss,ABTCCHF BW,,,,
Börse Stuttgart,,2BTC GS,,,
Börse Düsseldorf,,2BTC GD,,,


In [16]:
d = d[[10,11,12,13,14]].agg('-'.join, axis=1)

d = pd.DataFrame(d)

In [17]:
def f(x):
    x = x.split('-')
    x = [i[:4] for i in x if i.strip()!='']
    x = set(x)
    
    return list(x)[0]

In [18]:
d = d[0].apply(lambda x:f(x))

t = df[df['Product / Underlying']==name][list(df)[4:]]

t.set_index('Currency', inplace=True)

t = t.fillna(' ')
t = t.agg('-'.join, axis=1)

t = t.apply(lambda x:f(x))

t = t.to_dict()

In [19]:
tt = {}
for k,v in t.items():
    if v not in tt:
        tt[v] = [k]
    else:
        tt[v].append(k)

In [20]:
d = pd.DataFrame(d)

d[1] = d[0].apply(lambda x: ','.join(tt[x]))

d.reset_index(inplace=True)

df_3 = d

In [21]:
df_3

,index,0,1
0,SIX Swiss Exchange,ABTC,"CHF,GBP,USD,SEK"
1,Deutsche Börse Xetra,2BTC,EUR
2,Euronext Amsterdam,ABTC,"CHF,GBP,USD,SEK"
3,Euronext Paris,ABTC,"CHF,GBP,USD,SEK"
4,Wiener Börse,ABTC,"CHF,GBP,USD,SEK"
5,Nasdaq Stockholm,ABTC,"CHF,GBP,USD,SEK"
6,BX Swiss,ABTC,"CHF,GBP,USD,SEK"
7,Börse Stuttgart,2BTC,EUR
8,Börse Düsseldorf,2BTC,EUR


In [22]:
len(df_1) + len(df_2) + len(df_3)

19

In [23]:
import numpy as np
x = np.reshape([float('NaN')]*300,(50,6))
df_temp = pd.DataFrame(x)

In [24]:
row = df_1.shape[0]
col = df_1.shape[1]

In [25]:
df_temp.iloc[0:1 ,2:3] = ['Pre-trade analysis - 28.04.2022']

In [26]:
df_temp.iloc[1:2 ,1:2] = ['Details of the ETP']

In [27]:
df_temp.iloc[2:2+row,1:1+col] = df_1

In [28]:
row1 = df_2.shape[0]
col1 = df_2.shape[1]

In [29]:
df_temp.iloc[4+row: 5+row ,1:2] = ['Primary market data']

In [30]:
df_temp.iloc[4+row: 4+row+row1 ,1:1+col1] = df_2

In [31]:
row2 = df_3.shape[0]
col2 = df_3.shape[1]

In [32]:
df_temp.iloc[5+row+row1: 6+row+row1 ,1:2] = ['Trading information']

In [33]:
df_temp.iloc[6+row+row1: 7+row+row1 ,1:1+col2] = ['Exchange','Ticker','Currency']

In [34]:
df_temp.iloc[7+row+row1: 7+row+row1+row2 ,1:1+col2] = df_3

In [35]:
df_temp.iloc[9+row+row1+row2: 10+row+row1+row2 ,1:2] = ['Liquidity analysis']

In [36]:
df_temp.iloc[10+row+row1+row2: 16+row+row1+row2 ,1:2] = ['20D daily average spread',
'Primary market turnover 1M',
'Secondary market turnover 1M',
'Underlying turnove',
'Desired amount to trade',
'% wrt underlying liquidity',]

In [37]:
df_temp.to_excel('newfile_v1.xlsx',index=False,header=False)

In [38]:
df_temp

,0,1,2,3,4,5
0,NaN,NaN,Pre-trade analysis - 28.04.2022,NaN,NaN,NaN
1,NaN,Details of the ETP,NaN,NaN,NaN,NaN
2,NaN,Name,21Shares Bitcoin ETP,NaN,NaN,NaN
3,NaN,ISIN,CH0454664001,NaN,NaN,NaN
4,NaN,TER,NaN,NaN,NaN,NaN
5,NaN,Type,Single Asset,NaN,NaN,NaN
6,NaN,Underlying,Bitcoin,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Primary Exchange,SIX,NaN,NaN,NaN
